In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [2]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
# Preprocess the data
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [5]:
## Encode Categorical VAriable
label_encoder_gender = LabelEncoder()
df['Gender'] = label_encoder_gender.fit_transform(df['Gender'])
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [6]:
## OHE 'geography'

from sklearn.preprocessing import OneHotEncoder
ohe_geo = OneHotEncoder()
geo_encoded = ohe_geo.fit_transform(df[['Geography']])
geo_encoded.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [7]:
ohe_geo.get_feature_names_out()

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [8]:
geo_encoded_df = pd.DataFrame(geo_encoded.toarray(), columns=['Geography_France', 'Geography_Germany', 'Geography_Spain'])
geo_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [9]:
## Combine one hot encoded with the original data
df = df.drop('Geography', axis=1)
df = pd.concat([df, geo_encoded_df], axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [10]:
# Split the data into features and target

X = df.drop('EstimatedSalary', axis=1)
y = df['EstimatedSalary']

In [11]:
## split the data in training and testing set
X_train, X_test, y_train, y_test = train_test_split(X ,y , test_size=0.20, random_state=42)

#scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [13]:
## pickling

with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('ohe_geo.pkl', 'wb') as file:
    pickle.dump(ohe_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [15]:
## Train Ann with regression

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [22]:
model = Sequential((
    Dense(64, activation='relu', input_shape=(X_train.shape[1], )), #HL1 conected with input layer and only in first layer input is required
    Dense(32, activation = 'relu'), #HL2
    Dense(1) #output layer
))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 64)                832       
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                                 
 dense_11 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])

In [25]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 64)                832       
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                                 
 dense_11 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
## setup the tensor board
import datetime
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

log_dir = "reglogs/fit/" + datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
tensorflow_callback = TensorBoard(log_dir = log_dir, histogram_freq=1)


## Set Up Early Stopping
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience=10, restore_best_weights=True)


In [28]:
#Train the model
history = model.fit(
    X_train, y_train, validation_data = (X_test, y_test), epochs =100,
    callbacks=[tensorflow_callback, early_stopping_callback]
)

Epoch 1/100


250/250 [==============================] - 4s 6ms/step - loss: 100383.6484 - mae: 100383.6484 - val_loss: 98529.1094 - val_mae: 98529.1094
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 99626.1328 - mae: 99626.1328 - val_loss: 96966.5938 - val_mae: 96966.5938
Epoch 3/100
250/250 [==============================] - 1s 5ms/step - loss: 96891.5781 - mae: 96891.5781 - val_loss: 92952.6484 - val_mae: 92952.6484
Epoch 4/100
250/250 [==============================] - 1s 5ms/step - loss: 91507.1484 - mae: 91507.1484 - val_loss: 86294.3125 - val_mae: 86294.3125
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 83684.8672 - mae: 83684.8672 - val_loss: 77732.0078 - val_mae: 77732.0078
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 74551.0859 - mae: 74551.0859 - val_loss: 68838.5859 - val_mae: 68838.5859
Epoch 7/100
250/250 [==============================] - 1s 5ms/step - loss: 65708.0391 - mae: 65708.039

In [29]:
%load_ext tensorboard

In [33]:
%tensorboard --logdir reglogs/fit

Reusing TensorBoard on port 6006 (pid 5556), started 0:00:21 ago. (Use '!kill 5556' to kill it.)

In [35]:
## evaluate model on the test data
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE: {test_mae}')

63/63 [==============================] - 0s 4ms/step - loss: 50240.3281 - mae: 50240.3281
Test MAE: 50240.328125


In [36]:
model.save('reg_model.h5')

d:\aroo\AI ML\ML\26-ANN\Self\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
